***
#### Import of required libraries

In [ ]:
import glob
import warnings
from os.path import join as opj

from traffic.core import Traffic
import pandas as pd

warnings.simplefilter(action="ignore", category=FutureWarning)

***
#### Import of MeteoSchweiz T/RH/P data

In [ ]:
weather_data_path = "/mnt/beegfs/store/MIAR/raw/meteo/T-RH_QFE-SMN_KLO"
list_csv = glob.glob(opj(weather_data_path, "*.csv"))
df_weather = pd.concat(
    [
        pd.read_csv(
            f,
            sep=";",
            header=None,
            index_col=None,
            parse_dates=False,
            names=[
                "timestamp",
                "temperature_gnd",
                "humidity_gnd",
                "pressure_gnd",
            ],
        )
        for f in list_csv
    ],
    axis=0,
)
df_weather["timestamp"] = pd.to_datetime(
    df_weather["timestamp"], format="%d.%m.%Y %H:%M:%S", utc=True
)

***
#### Import of MeteoSchweiz wind data

In [ ]:
wind_data_path = "/mnt/beegfs/store/MIAR/raw/meteo/Wind_LSZH_2018-2023"
list_csv_C = glob.glob(
    opj(wind_data_path, "**/*Kloten_Wind_C*.csv.zip"), recursive=True
)


def read_wind(list_csv: list) -> pd.DataFrame:
    df_wind = pd.concat(
        [
            pd.read_csv(
                f,
                sep=";",
                header=None,
                index_col=0,
                parse_dates=True,
                names=["date", "wind_speed_gnd", "wind_direction_gnd"],
                usecols=[0, 1, 2],
            )
            for f in list_csv
        ],
        axis=0,
    ).sort_index()
    df_wind = df_wind.reset_index().rename(columns={"date": "timestamp"})
    df_wind["timestamp"] = df_wind["timestamp"].dt.tz_localize("UTC")
    return df_wind


wind_28 = read_wind(list_csv_C)

***
#### Import of FZAG mass/typecode data

In [ ]:
fzag_data_path = "/mnt/beegfs/store/MIAR/raw/FZAG"
df_departures = pd.read_csv(
    f"{fzag_data_path}/df_departure.csv",
    sep=",",
    header=0,
    index_col=0,
)
df_departures = (
    pd.read_csv(
        f"{fzag_data_path}/df_departure.csv", sep=",", header=0, index_col=0
    )
    .rename(
        columns={
            "SDT": "date",
            "CSG": "callsign",
            "TWT": "toff_weight_kg",
            "ITY": "typecode",
        }
    )
    .drop(columns=["REG"])
)
df_departures["date"] = pd.to_datetime(df_departures["date"])

***
#### Import of ADS-B data

In [ ]:
t = Traffic.from_file(
    "/mnt/beegfs/store/MIAR/raw/adsb_osn/LSZH/05_category_runway_overall/"
    "departure_28.parquet"
)

In [ ]:
t = Traffic.from_file(
    "/mnt/beegfs/store/MIAR/raw/adsb_osn/LSZH/05_category_runway_overall/"
    "departure_28.parquet"
)


def after_aligned(flight):
    try:
        t_start = flight.aligned_on_runway("LSZH")[0].stop
        flight.data = flight.data[flight.data.timestamp > t_start]
        if len(flight) > 200:
            return flight
    except:
        pass


t.data["date"] = pd.to_datetime(t.data["timestamp"].dt.date)

t = (
    t.iterate_lazy()
    .pipe(after_aligned)
    .filter("aggressive")
    .query("altitude < 18000")
    .query("date <= '2023-09-30'")
    .resample("1s")
    .eval(desc="processing", max_workers=25)
)

***
#### Merging of data

In [ ]:
# FZAG mass/typecode data
t.data = pd.merge(
    left=t.data,
    right=df_departures,
    left_on=["date", "callsign"],
    right_on=["date", "callsign"],
    how="left",
)

# MeteoSchweiz wind data
t.data = pd.merge_asof(
    left=t.data.sort_values("timestamp"),
    right=wind_28.sort_values("timestamp"),
    on="timestamp",
    tolerance=pd.Timedelta("30min"),
)

# MeteoSchweiz T/RH/P data
t.data = pd.merge_asof(
    left=t.data.sort_values("timestamp"),
    right=df_weather.sort_values("timestamp"),
    on="timestamp",
    tolerance=pd.Timedelta("30min"),
)

# Only keep flights with complete weather data
t = t.query(
    "wind_speed_gnd.notna() and wind_direction_gnd.notna() and "
    "temperature_gnd.notna() and humidity_gnd.notna() and pressure_gnd.notna()"
)

# Create an additional dataset only containing flights with mass data
t_w = t.query("toff_weight_kg.notna()")

***
#### Data export

In [ ]:
output_dir = "/mnt/beegfs/store/MIAR/merged/osn"
t.to_parquet(f"{output_dir}/takeoffs28_all_complete.parquet")
t_w.to_parquet(f"{output_dir}/takeoffs28_mass_complete.parquet")